### ユーザー名からタイムラインのツイートデータを取得してCSVに書き起こすプログラム

In [168]:
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd
import os
from dotenv import load_dotenv

##### Twitter認証（ここはTwitterAPIを使うとき大体共通）

In [169]:
# Twitter情報

load_dotenv()

# API情報
CK = os.environ['CONSUMER_KEY']
CS = os.environ['CONSUMER_SECRET']
AT = os.environ['ACCESS_TOKEN']
ATS = os.environ['ACCESS_TOKEN_SECRET']

#Twitterの認証
auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AT, ATS)
api = tweepy.API(auth)

#　”wait_on_rate_limit = True”　利用制限にひっかかた時に必要時間待機する
api=tweepy.API(auth,wait_on_rate_limit=True)

##### ↓次のセルを変更
- Account...@から始まるアカウント名を指定
- req_tweet_cnt...取得するツイート数を指定
  - 1500 requests / 15 mins PER APP
  - 900 requests / 15 mins PER USER

In [170]:
#取得するツイートの数と、検索するアカウントの入力
Account= '@usadapekora'
req_tweet_cnt = 4000

##### 必要なTweetデータを取得
api.user_timeline： 特定のユーザーのタイムライン情報を取得

``` python
tweepy.Cursor(api.user_timeline, id="アカウント名").items(取得上限)
```

In [171]:
#検索条件を元にツイートを抽出
tweets = tweepy.Cursor(api.user_timeline, tweet_mode='extended', id=Account).items(req_tweet_cnt)

##### 取得できる時間データは世界時間なので日本時間に形式を変換

In [172]:
#世界時間を日本時間に変換する
def change_time_JST(time):
    utc = datetime(time.year, time.month,time.day, \
    time.hour,time.minute,time.second, tzinfo=timezone.utc)
    jst = utc.astimezone(pytz.timezone("Asia/Tokyo"))
    str_time = jst.strftime("%Y-%m-%d_%H:%M:%S")
    return str_time

##### Tweetデータを格納

In [173]:
# 取得したデータを格納する配列
tweet_data = []

poems = {}

# データを格納するイテレーション
for tweet in tweets:
    #ツイート時刻とユーザのアカウント作成時刻を日本時刻にする
    tweet_time = change_time_JST(tweet.created_at)
    # create_account_time = change_time_JST(tweet.user.created_at)

    # リツイートとリプライを除外しながらデータを取得
    # if not tweet.favorite_count == 0 and not "@" in tweet.text:

    # 引用ツイートかつhttp~の配信URLを持つツイート内容をポエムとして保持
    if tweet.is_quote_status and ('http' in tweet.full_text):
        poems[tweet.quoted_status_id_str] = tweet.full_text.split('http')[0]
        continue

    # "▽待機所▽"が含まれるツイートは配信前ツイートと判断して一覧を作る。また、そのツイートIDと保持していたポエムの引用元ツイートIDが一致したとき、ポエムをCSVに書き込む
    if "▽待機所▽" in tweet.full_text:
        #tweet_dataの配列に取得したい情報を入れていく
        tweet_data.append([
            tweet.id,
            tweet_time,
            tweet.full_text,
            poems[tweet.id_str] if (tweet.id_str in poems) else ""
            # tweet.favorite_count,
            # tweet.retweet_count,
            # tweet.user.screen_name,
            # tweet.user.name,
            # tweet.user.description,
            # tweet.user.friends_count,
            # tweet.user.followers_count,
            # create_account_time,
            # tweet.in_reply_to_status_id_str,
            # tweet.in_reply_to_user_id_str,
            # tweet.coordinates,
            # tweet.user.verified
        ])

Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id


##### 取り出したデータをpandasのDataFrameに変換

In [174]:
#CSVファイルに出力するときの列の名前を定義
columns=[
    'ツイートID',
    'ツイートされた時間(日本時間)',
    '配信前ツイート',
    # 'いいねの数',
    # 'リツイートの数',
    # 'ツイートした人のID',
    # 'アカウント名',
    # '自己紹介文',
    # 'フォロー数',
    # 'フォロワー数',
    # 'アカウント作成日時',
    # 'ツイートが返信の場合、返信先のツイートID',
    # 'ツイートが返信の場合、返信先のユーザーID',
    # 'ツイートされた場所',
    # '認証アカウント(公式アカウント)かどうか'
    'ぽえーむ'
    ]

#tweet_dataのリストをpandasのDataFrameに変換
df = pd.DataFrame(tweet_data, columns=columns)

##### CSVファイルに出力する

In [175]:
#CSVファイルの名前を決める
csv_file_name='pekora_poems2.csv'

#CSVファイルを出力する
df.to_csv('./output/'+csv_file_name,encoding='utf-8-sig',index=False)

##### テキストファイルに出力する

In [176]:
# テキストファイルの名前を決める
text_file_name = 'pekora_poems2.txt'

# 'ぽえーむ'の列の要素を抽出してリスト化、その後区切り文字','を指定して文字列結合
text = ','.join(df['ぽえーむ'].to_list())

# テキストファイルを出力する
with open('./output/'+text_file_name, 'w') as f:  # 「with open('test.txt', 'wt') as f:」と同じ
    f.writelines(text)  # 文字列リストはwritelinesメソッドで書き込む